In [2]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 8.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 20.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'March-Mania-2023/MNCAA_train.csv'
file_key_2 = 'March-Mania-2023/MNCAA_test.csv'
file_key_3 = 'March-Mania-2023/WNCAA_train.csv'
file_key_4 = 'March-Mania-2023/WNCAA_test.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data files
man_train = pd.read_csv(file_content_stream_1)
man_test = pd.read_csv(file_content_stream_2)
woman_train = pd.read_csv(file_content_stream_3)
woman_test = pd.read_csv(file_content_stream_4)

# Man

In [11]:
man_test.head()

,ID,Pred,Season,T1,T2,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2
0,2023_1101_1102,0.5,2023,1101,1102,NaN,72.090909,68.5,-2.863636,57.772727,57.0,48,72,14.818182,1.954545,51.045455,43,0.510983,71.256818,1.78142,0.23809,0.166512,0.758653,NaN,67.517241,67.0,0.241379,50.689655,52.0,41,61,15.448276,4.103448,53.551724,44,0.552858,63.612931,1.590323,0.232264,0.167316,0.713868,NaN,NaN,NaN,NaN,NaN
1,2023_1101_1103,0.5,2023,1101,1103,NaN,72.090909,68.5,-2.863636,57.772727,57.0,48,72,14.818182,1.954545,51.045455,43,0.510983,71.256818,1.78142,0.23809,0.166512,0.758653,NaN,72.038462,72.5,5.653846,57.461538,57.0,45,71,12.615385,2.615385,55.846154,46,0.508348,68.261538,1.706538,0.249010,0.158281,0.744768,NaN,NaN,NaN,NaN,NaN
2,2023_1101_1104,0.5,2023,1101,1104,NaN,72.090909,68.5,-2.863636,57.772727,57.0,48,72,14.818182,1.954545,51.045455,43,0.510983,71.256818,1.78142,0.23809,0.166512,0.758653,NaN,83.444444,81.0,15.111111,61.777778,63.0,48,90,15.333333,5.074074,64.851852,54,0.539098,75.779630,1.894491,0.285735,0.179910,0.747750,NaN,NaN,NaN,NaN,NaN
3,2023_1101_1105,0.5,2023,1101,1105,NaN,72.090909,68.5,-2.863636,57.772727,57.0,48,72,14.818182,1.954545,51.045455,43,0.510983,71.256818,1.78142,0.23809,0.166512,0.758653,NaN,69.200000,70.0,-3.560000,57.880000,59.0,43,68,12.320000,3.880000,55.600000,47,0.489690,72.298000,1.807450,0.230658,0.185098,0.665295,NaN,NaN,NaN,NaN,NaN
4,2023_1101_1106,0.5,2023,1101,1106,NaN,72.090909,68.5,-2.863636,57.772727,57.0,48,72,14.818182,1.954545,51.045455,43,0.510983,71.256818,1.78142,0.23809,0.166512,0.758653,NaN,63.777778,62.0,-9.703704,59.185185,59.0,44,76,10.000000,3.481481,59.518519,49,0.429708,69.919444,1.747986,0.239261,0.170748,0.656706,NaN,NaN,NaN,NaN,NaN


In [4]:
man_train['target'] = np.where(man_train['ResultDiff'] > 0, 1, 0)
man_train.head()

,Season,DayNum,T1,T2,T1_Points,T2_Points,ResultDiff,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,target
0,2003,134,1421,1411,92,84,8,1.000000,71.206897,71.0,-7.241379,56.793103,57.0,45,76,13.034483,3.000000,60.965517,50,0.489800,70.666379,1.766659,0.287716,0.228101,0.697730,0.833333,72.800000,72.5,1.966667,55.266667,55.0,39,73,14.200000,2.233333,60.400000,43,0.503036,70.665000,1.766625,0.330188,0.215143,0.724640,16,16,0,0.866989,0.974984,1
1,2003,136,1112,1436,80,51,29,0.666667,85.214286,86.5,14.964286,65.714286,66.0,50,88,17.642857,4.214286,64.678571,52,0.517632,77.196429,1.929911,0.272214,0.190130,0.724040,1.000000,67.793103,65.0,4.655172,55.862069,58.0,44,67,14.206897,2.965517,55.068966,44,0.494732,66.252586,1.656315,0.235084,0.212863,0.802653,1,16,-15,1.193799,0.975313,1
2,2003,136,1113,1272,84,71,13,0.666667,75.965517,75.0,6.793103,56.896552,57.0,46,69,15.551724,4.241379,55.655172,43,0.517334,69.655172,1.741379,0.313270,0.200649,0.764390,0.750000,74.517241,73.0,8.689655,60.000000,61.0,50,79,16.620690,5.068966,57.862069,42,0.498337,70.600000,1.765000,0.259300,0.194125,0.765180,10,7,3,0.862123,1.049829,1
3,2003,136,1141,1166,79,73,6,1.000000,79.344828,80.0,6.103448,52.689655,53.0,40,63,15.620690,4.000000,57.931034,44,0.572835,72.301724,1.807543,0.369588,0.252913,0.752266,1.000000,79.242424,80.0,14.909091,57.454545,57.0,48,69,16.818182,4.454545,55.333333,45,0.567455,69.453788,1.736345,0.245127,0.192564,0.744973,11,6,5,NaN,1.074422,1
4,2003,136,1143,1301,76,74,2,0.333333,74.482759,75.0,4.724138,58.724138,58.0,42,77,16.000000,2.793103,58.793103,48,0.524098,70.925862,1.773147,0.240967,0.198374,0.736381,0.600000,72.400000,71.5,4.400000,53.333333,52.0,44,73,14.666667,3.066667,53.133333,35,0.534189,67.521667,1.688042,0.298489,0.209940,0.755463,8,9,-1,0.979448,0.894431,1


In [8]:
man_train.shape

(2496, 49)

In [6]:
man_train['Season'].value_counts()

2012    134
2013    134
2019    134
2018    134
2017    134
2016    134
2015    134
2014    134
2022    134
2011    134
2021    132
2004    128
2010    128
2009    128
2008    128
2007    128
2006    128
2005    128
2003    128
Name: Season, dtype: int64

# Help Funs

In [ ]:
def modeling(train, test, model):

# Cross Validation

In [10]:
start_year = 2010
end_year = 2021

mse_scores = list()

for i in range(start_year, end_year):
    
    train_data = man_train[man_train['Season'] <= start_year].reset_index(drop = True)
    test_data = man_train[man_train['Season'] == (start_year + 1)].reset_index(drop = True)
    
    modeling_results = modeling(train_data, test_data)
    mse_scores.append(modeling_results[0])